In [259]:
%%javascript
// Disable scrolling on plots
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [260]:
%matplotlib notebook

In [261]:
import pandas as pd
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt

from ipywidgets import interact, interact_manual, interactive, fixed, interact_manual
import ipywidgets as widgets

from IPython.display import display
from IPython.core.display import display, HTML

In [262]:
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", 1000)

In [263]:
# Make cells wider
display(HTML("<style>.container { width:90% !important; }</style>"))

In [264]:
RESULTS_DIR = Path("./results")
METHODS = ("qsort_c", "vanilla_quicksort", "insertion_sort")
DEFAULT_METHOD = METHODS[0]
INPUT_TYPES = ("ascending", "descending", "random", "single_num", "N/A")

dirs = list(RESULTS_DIR.iterdir())
try:
    LATEST_RESULT = dirs[-1]
except IndexError as e:
    raise IndexError("No results found") from e

csvs = list(LATEST_RESULT.glob("*.csv"))
if len(csvs) < 1:
    raise KeyError("CSV missing")
IN_FILE = csvs[0]

df = pd.read_csv(IN_FILE)

# Convert microseconds to milliseconds and rename column
df["Elapsed Time (microseconds)"] = df["Elapsed Time (microseconds)"] / 1000
df.rename(
    columns={"Elapsed Time (microseconds)": "Runtime (milliseconds)"}, inplace=True
)
# Add seconds column
df["Runtime (seconds)"] = df["Runtime (milliseconds)"] / 1000

# Reorder columns
df = df[
    [
        "Input",
        "Description",
        "Method",
        "Size",
        "Threshold",
        "Runtime (milliseconds)",
        "Runtime (seconds)",
        "Valid",
    ]
]

# Average runtime for repeated runs.
df = (
    df.groupby(["Input", "Method", "Description", "Size", "Threshold"])
    .mean()
    .reset_index()
)

# Get min and max threshold
default_df = df[df["Method"] == DEFAULT_METHOD]
MIN_THRESHOLD = default_df["Threshold"].min()
MAX_THRESHOLD = default_df["Threshold"].max()

In [272]:
def plot_vs_methods(method=DEFAULT_METHOD, method_2=DEFAULT_METHOD, thresh=4, unit="milliseconds"):
    threshold = 0 if method != "qsort_c" else thresh
    threshold_2 = 0 if method_2 != "qsort_c" else thresh

    method_dfs = []
    method_2_dfs = []

    for i in INPUT_TYPES:
        m = df[df["Method"] == method]
        m2 = df[df["Method"] == method_2]

        d = m[m["Description"] == i]
        d2 = m2[m2["Description"] == i]

        method_dfs.append(d[d["Threshold"] == threshold])
        method_2_dfs.append(d2[d2["Threshold"] == threshold_2])

    for frame, frame2 in zip(method_dfs, method_2_dfs):
        # Skip if empty frame
        if not len(frame) or not len(frame2):
            continue

        # Sort for pretty graphs
        frame = frame.sort_values(["Description", "Method", "Threshold", "Size"])
        frame2 = frame2.sort_values(["Description", "Method", "Threshold", "Size"])

        # Select the first row to get some more info
        row_0 = frame.iloc[0]

        # Assemble the title
        title = ""
        if method == "qsort_c":
            title += f"{method} (threshold: {threshold})"
        else:
            title += method
        title += " vs. "
        if method_2 == "qsort_c":
            title += f"{method_2} (threshold: {threshold_2})"
        else:
            title += method_2
        title += f"\n{row_0['Description']}"

        # Plot
        ax = frame.plot(
            x="Size", y=f"Runtime ({unit})", marker=".", markersize=10, title=title
        )
        frame2.plot(
            ax=ax,
            x="Size",
            y=f"Runtime ({unit})",
            marker=".",
            markersize=10,
        )

        # Fix legends to method names.
        ax.legend([method, method_2])
        
        # Set axis titles
        ax.set_xlabel("Size")
        ax.set_ylabel(f"Runtime ({unit})")

        # Table of data
        display(frame.head())
        display(frame2.head())

In [276]:
def plot_vs_thresholds(threshold=4, threshold_2=4, unit="milliseconds"):
    pass

In [277]:
interact_manual(
    plot_vs_methods,
    name="Plot",
    method=list(METHODS),
    method_2=list(METHODS),
    unit=["milliseconds", "seconds"],
    thresh=widgets.IntSlider(
        min=MIN_THRESHOLD, max=MAX_THRESHOLD, step=1, value=MIN_THRESHOLD
    ),
    continuous_update=False,
)

interactive(children=(Dropdown(description='method', options=('qsort_c', 'vanilla_quicksort', 'insertion_sort'…

<function __main__.plot_vs_methods(method='qsort_c', method_2='qsort_c', thresh=4, unit='milliseconds')>